In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import datetime
import csv
%matplotlib inline


# Import Garmin Data

In [36]:
def get_data(filename):
    reader = csv.DictReader(open(chi_marathon, 'rb'))
    chicago = []
    for row in reader:
        for key, value in row.iteritems():
            if key in ['Distance', 'Avg HR', 'Elev Gain']:
                row[key] = float(row[key])
                data.append(row)
        return chicago

In [37]:
chicago.head(5)

,Activity Type,Date,Title,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,Avg Run Cadence,...,Grit,Flow,Climb Time,Bottom Time,Min Temp,Surface Interval,Decompression,Best Lap Time,Number of Runs,Max Temp
0,Treadmill Running,5/3/19,Treadmill Running,1.30,103,0:11:12,158,185,2.3,156,...,0,0,0:00,0:00,77.0,0:00,No,04:02.7,2,0
1,Treadmill Running,5/6/19,Treadmill Running,1.16,71,08:54.2,139,154,1.6,164,...,0,0,0:00,0:00,77.0,0:00,No,01:37.7,2,0
2,Running,5/15/19,Chicago Running,2.00,135,0:16:40,142,159,2.2,172,...,0,0,0:00,0:00,68.0,0:00,No,00:01.1,3,0
3,Running,5/15/19,Chicago Running,1.01,80,08:49.4,152,164,2.1,167,...,0,0,0:00,0:00,73.4,0:00,No,00:03.6,2,0
4,Treadmill Running,5/16/19,Treadmill Running,1.01,73,07:58.7,155,162,2.1,171,...,0,0,0:00,0:00,75.2,0:00,No,00:04.9,2,0


# Examining the chicago marathon training data

Total miles ran while training for the Chicago Marathon (includes marathon)

In [38]:
total_distance = chicago['Distance'].sum(skipna=True)
print("Total Distance ran", total_distance, "miles")

Total Distance ran 512.77 miles


Furthest Distance Ran

In [39]:
chicago['Distance'].max()

26.49

Times for max distances

In [40]:
chicago.groupby(['Time'])['Distance'].max()

Time
01:46.4     0.13
02:27.3     0.31
02:31.5     0.31
02:43.9     0.31
03:51.8     0.45
           ...  
1:58:02    15.02
1:58:49    14.01
2:33:27    18.01
2:38:12    20.02
3:47:34    26.49
Name: Distance, Length: 117, dtype: float64

Add pace

In [44]:
pace = chicago.Time / chicago.Distance
chicago = pd.chicago(pace)

TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [42]:
chicago.head()

,Activity Type,Date,Title,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,Avg Run Cadence,...,Grit,Flow,Climb Time,Bottom Time,Min Temp,Surface Interval,Decompression,Best Lap Time,Number of Runs,Max Temp
0,Treadmill Running,5/3/19,Treadmill Running,1.30,103,0:11:12,158,185,2.3,156,...,0,0,0:00,0:00,77.0,0:00,No,04:02.7,2,0
1,Treadmill Running,5/6/19,Treadmill Running,1.16,71,08:54.2,139,154,1.6,164,...,0,0,0:00,0:00,77.0,0:00,No,01:37.7,2,0
2,Running,5/15/19,Chicago Running,2.00,135,0:16:40,142,159,2.2,172,...,0,0,0:00,0:00,68.0,0:00,No,00:01.1,3,0
3,Running,5/15/19,Chicago Running,1.01,80,08:49.4,152,164,2.1,167,...,0,0,0:00,0:00,73.4,0:00,No,00:03.6,2,0
4,Treadmill Running,5/16/19,Treadmill Running,1.01,73,07:58.7,155,162,2.1,171,...,0,0,0:00,0:00,75.2,0:00,No,00:04.9,2,0


Given a distance and time, computes your pace-per-mile:

if len(sys.argv) == 1:
    print('Usage: <distance in mi> <time>')
    sys.exit()
    distance = sys.argv[1]

total_time = sys.argv[2]
if total_time.count(':') == 2:
    hours, minutes, seconds = total_time.split(':')
else:
    hours = 0
    minutes, seconds = total_time.split(':')

total_seconds = (int(hours) * 3600) + (int(minutes) * 60) + int(seconds)
seconds_per_mile = float(total_seconds) / float(distance)

minutes_per_mile = int(seconds_per_mile / 60)
seconds_remainder = int(seconds_per_mile - (minutes_per_mile * 60))
print ('Avg. Pace of {}:{:0=2d}'.format(minutes_per_mile, seconds_remainder))

# Linear Prediction
Predicted time = Desired distance * Pace

In [ ]:
linear time prediction based on distance 

In [34]:
def linear(goal_distance, pace):
    predicted_time = paced_time(goal_distance, time_to_seconds(pace))
    return predicted_time

Using Runner's world time prediction model 
https://www.runnersworld.com/uk/training/a761681/rws-race-time-predictor/

Predicted time= T1 * (D2 / D1)^C

T1 is the time achieved for D1

D1 is the distance over which the initial time is achieved

D2 is the distance for which the time is to be predicted

C is the pace degradation coefficient, from 1.06 to 1.10

In [35]:
def riegel(goal_distance, distance, pace, degradation_coefficient):
    time_in_seconds = paced_time(distance, time_to_seconds(pace))
    predicted_time = time_seconds * ( goal_distance / distance ) ** degradation_coefficient
    return predicted_time

#Part of this was forked form github